In [1]:
import pandas as pd
import numpy as np

# Find representative producer countries

In [2]:
df = pd.read_excel('imap_export.xls', header=1)
df

,Factory Name,Factory Type,Product Type Type,"Nike, Inc. Brand(s)",Events,Supplier Group,Address,City,State,Postal Code,Country / Region,Region,Total Workers,Line Workers,% Female Workers,% Migrant Workers
0,"A & K Designs, Inc.",FINISHED GOODS,Apparel,Nike,NaN,A & K DESIGNS,8564 NE Alderwood Road,Portland,Oregon,97220,USA,AMERICAS,111,95,73,0
1,"ACode Sporting Goods Co., Ltd.",FINISHED GOODS,Equipment,Nike,NaN,EXCELLENCE SPORTING GOODS,No 32 VSIP II A Street 31,Bac Tan Uyen,Bình Duong,822710,Vietnam,SE ASIA,347,318,79,0
2,Ad Dulyal,FINISHED GOODS,Apparel,Nike,NaN,MAS HOLDINGS,Part of land no 1075 Basin 5 Ad Dulayl,Zarqa,Az Zarqa,11183,Jordan,EMEA,1303,1166,87,75
3,ADORA FOOTWEAR LIMITED,FINISHED GOODS,Footwear,Converse,NaN,HUALI,TAM DIEP INDUSTRY ZONE,Ninh Binh Province,Ninh Bình,430000,Vietnam,SE ASIA,8120,6800,86,0
4,"AHP APPAREL PVT LTD., UNIT 60",FINISHED GOODS,Apparel,Nike,Collegiate,SHAHI,207 ABDE F KIADB INDUSTRIAL AREA,HASSAN,Karnataka,573201,India,S ASIA,2704,906,75,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
638,LELABPLUS,FINISHED GOODS,Apparel,Nike,NaN,LELAB,1 bis rue Jean le Galleu,Ivry sur Seine,Île-de-France,94200,France,EMEA,21,14,62,100
639,"Haivina Co., Ltd.",FINISHED GOODS,Equipment,Nike,NaN,HAIVINA,Lang Xuyen village,Hai Duong province,Hi Duong,170000,Vietnam,SE ASIA,3259,2870,96,0
640,Tower Garments (London) Ltd,FINISHED GOODS,Apparel,Nike,NaN,TOWER GARMENTS,Unit J 17 Queensway Enfield,ENFIELD,London-- City of,EN3 4SA,United Kingdom,EMEA,90,90,60,0
641,"QINGDAO HONGTAISHENGDA TRADE CO., LTD.",FINISHED GOODS - COMPONENTS,Footwear,Nike,NaN,"QINGDAO HONGTAISHENGDA TRADE CO., LTD.",Beijingxi Road 41,Qingdao,Shandong,266300,China,N ASIA,100,80,90,0


In [3]:
workers_per_country = df.groupby(['Region', 'Country / Region']).sum()['Total Workers'].reset_index().sort_values(['Region', 'Total Workers'], ascending=False)
workers_per_country

,Region,Country / Region,Total Workers
37,SE ASIA,Vietnam,509169
35,SE ASIA,Cambodia,56673
36,SE ASIA,Thailand,26432
31,S ASIA,Indonesia,279988
33,S ASIA,Pakistan,44785
30,S ASIA,India,43407
34,S ASIA,Sri Lanka,38311
32,S ASIA,Malaysia,7508
25,N ASIA,China,143361
29,N ASIA,Taiwan,12714


In [4]:
# Representative producer countries for each region
top_countries = workers_per_country.groupby('Region').head(1)
top_countries

,Region,Country / Region,Total Workers
37,SE ASIA,Vietnam,509169
31,S ASIA,Indonesia,279988
25,N ASIA,China,143361
12,EMEA,Egypt,12462
1,AMERICAS,Brazil,21376


# Find representative consumer countries for each region
- For top countries by consumption, we choose countries by GDP (https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal), sorted by forecast).
    - North America - US, EMEA - Germany, Greater China - China, Asia Pacific - Japan, Latin America - Brazil
- Ports for reference: largest port in the country (UN/LOCODE)
    - US: Port of Los Angeles (USLAX)
    - China: Port of Shanghai (CNSGH / CNSHA)
    - Germany: Hamburg Port (DEHAM)
    - India: Mundra (INMUN)
    - Japan: Tokyo (JPTYO)
    - Vietnam: Hai Phong (VNSIT)
    - Indonesia: Tanjung Priok (IDTPR / IDTPP)
    - Egypt: Port Said (EGPSD)
    - Brazil: Santos (BRSSZ)

# Estimating transport cost (shipping + trucking cost) from producer region to consumer region
- We use the Drewry World Container Index (https://en.macromicro.me/collections/4356/freight/44756/drewry-world-container-index) to get the overall trend of shipping cost.
- Drewry tracks the freight costs of 40-foot container via eight major routes, including spot rates and short-term contract rates.
- We take the cost in the first week of July, which is a good proxy for the average cost. 
- Because CMA-CGM and Freightos shipping costs are not always available between countries, we take a distance metric instead (https://www.geodatos.net/en/distances/countries).
- We see that the Drewry World Container Index is very similar to the Drewry World Container Index - Shanghai to LA, and so we use that as a basis.
- I checked the cost from Shanghai to LA on Freightos for a forty-foot and the cost is (exepctedly) very similar, so we can use the index directly.

In [5]:
years = np.arange(2014, 2024)
# Get Drewry World Container Index cost
global_shipping_costs = pd.DataFrame(
    {'year': years, # 10 years, 2014-2023
     'global_shipping_cost': [2053.72, 1583.27, 1427.14, 1549.72, 1468.18, 1372.11, 2031.57, 8399.09, 7050.94, 1474.32]})
global_shipping_costs

,year,global_shipping_cost
0,2014,2053.72
1,2015,1583.27
2,2016,1427.14
3,2017,1549.72
4,2018,1468.18
5,2019,1372.11
6,2020,2031.57
7,2021,8399.09
8,2022,7050.94
9,2023,1474.32


In [6]:
# Get distances (km)
data = {
    # Rows: ['SE ASIA - Vietnam', 'S ASIA - Indonesia', 'N ASIA - China', 'EMEA - Egypt', 'AMERICAS - Brazil']
    # Columns:
    'Asia Pacific': [3864, 4808, 3054, 9704, 17371], # 'Asia Pacific - Japan'
    'Greater China': [2450, 4181, 0, 6896, 16638], # 'Greater China - China'
    'EMEA': [9339, 11014, 7242, 3203, 9417], # 'EMEA - Germany'
    'North America': [13814, 14972, 11671, 10997, 7301], # 'North America - US'
    'Latin America': [17899, 17740, 16638, 10017, 0] # 'Latin America - Brazil'
}

# Rows (representative producer country)
# ['SE ASIA - Vietnam', 'S ASIA - Indonesia', 'N ASIA - China', 'EMEA - Egypt', 'AMERICAS - Brazil']
rows = ['SE ASIA', 'S ASIA', 'N ASIA', 'EMEA', 'AMERICAS']

# Create df
distances = pd.DataFrame(data, index=rows)
distances

,Asia Pacific,Greater China,EMEA,North America,Latin America
SE ASIA,3864,2450,9339,13814,17899
S ASIA,4808,4181,11014,14972,17740
N ASIA,3054,0,7242,11671,16638
EMEA,9704,6896,3203,10997,10017
AMERICAS,17371,16638,9417,7301,0


In [7]:
# Normalize distances such that the distance from China to US is 1
transport_costs = distances / 11671
transport_costs = transport_costs.reset_index().melt(id_vars=['index'], var_name='Column', value_name='Value')
transport_costs = transport_costs.rename(columns={'index':'producer_region', 'Column':'consumer_region', 'Value':'distance'})
transport_costs

,producer_region,consumer_region,distance
0,SE ASIA,Asia Pacific,0.331077
1,S ASIA,Asia Pacific,0.411961
2,N ASIA,Asia Pacific,0.261674
3,EMEA,Asia Pacific,0.831463
4,AMERICAS,Asia Pacific,1.488390
5,SE ASIA,Greater China,0.209922
6,S ASIA,Greater China,0.358238
7,N ASIA,Greater China,0.000000
8,EMEA,Greater China,0.590866
9,AMERICAS,Greater China,1.425585


In [8]:
# Estimate number of items that can fit in a 40-foot (40') container
# A Nike shoe box measures 14 x 7.5 x 5 inches (14" x 7.5" x 7") (https://www.kusashoes.com/how-big-is-a-nike-shoe-box/)
# Using this calculator (https://www.gigacalculator.com/calculators/container-loading-calculator.php), 
# we get an estimate of 5,148 shoe boxes per container.
quantity_per_container = 5148

# Estimate factor for trucking cost from supplier to port of origin and from port of destination to customer
# As an estimate, we assume each direction costs 50% of shipping costs, so we have a factor of 2
trucking_cost_factor = 2

In [9]:
# Multiply by global_shipping_cost to get the total shipping cost (for each year)
# Then divide by 5,148 to get the shipping cost per item (for each year)
# And multiple by a factor of 2 to factor in trucking cost (for each year)
for i, year in enumerate(years):
    transport_costs[f'shipping_cost_{str(year)}'] = transport_costs['distance'] * global_shipping_costs.iloc[i,1] / quantity_per_container * trucking_cost_factor
transport_costs

,producer_region,consumer_region,distance,shipping_cost_2014,shipping_cost_2015,shipping_cost_2016,shipping_cost_2017,shipping_cost_2018,shipping_cost_2019,shipping_cost_2020,shipping_cost_2021,shipping_cost_2022,shipping_cost_2023
0,SE ASIA,Asia Pacific,0.331077,0.264157,0.203646,0.183564,0.199330,0.188843,0.176486,0.261308,1.080321,0.906917,0.189632
1,S ASIA,Asia Pacific,0.411961,0.328692,0.253398,0.228410,0.248028,0.234978,0.219602,0.325147,1.344250,1.128483,0.235961
2,N ASIA,Asia Pacific,0.261674,0.208782,0.160956,0.145084,0.157545,0.149256,0.139489,0.206531,0.853856,0.716802,0.149880
3,EMEA,Asia Pacific,0.831463,0.663400,0.511433,0.461000,0.500596,0.474257,0.443224,0.656245,2.713104,2.277620,0.476240
4,AMERICAS,Asia Pacific,1.488390,1.187543,0.915510,0.825230,0.896110,0.848961,0.793409,1.174735,4.856691,4.077136,0.852511
5,SE ASIA,Greater China,0.209922,0.167491,0.129123,0.116390,0.126387,0.119737,0.111902,0.165684,0.684986,0.575038,0.120238
6,S ASIA,Greater China,0.358238,0.285828,0.220353,0.198623,0.215683,0.204335,0.190964,0.282745,1.168950,0.981320,0.205190
7,N ASIA,Greater China,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,EMEA,Greater China,0.590866,0.471435,0.363442,0.327603,0.355741,0.337023,0.314970,0.466350,1.928026,1.618556,0.338433
9,AMERICAS,Greater China,1.425585,1.137433,0.876879,0.790408,0.858297,0.813137,0.759930,1.125165,4.651754,3.905094,0.816538


### Comments
The transport cost is typically $0.20-$1.00 per pair of shoes, but spiked to $1.00-$4 during Covid-19 (2021-2022). Looks reasonable.

In [10]:
# Save as csv file
transport_costs.to_csv('transport_costs.csv', index=False)